In [1]:
import glob

In [2]:
list_of_txts=glob.glob("C:/anaconda/ep/ep*.txt")

In [3]:
import random

In [4]:
random.seed(1)
s=random.sample(range(0,len(list_of_txts)), round(0.8*(len(list_of_txts))))

In [5]:
train_txts= [list_of_txts[i] for i in s]

In [6]:
 t=list(set(range(0,len(list_of_txts))) - set(s))

In [7]:
test_txts= [list_of_txts[i] for i in t]

In [8]:
content=[]
for file in train_txts:
    fp=open(file, encoding="utf8")
    try:
        for line in fp:
             content.append(line)
    finally:
        fp.close()


In [9]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [10]:
no_html_content=[]
for s in content:
     no_html_content.append(cleanhtml(s))


In [11]:
content=[]
for s in no_html_content:
    if s!='\n':
        content.append(s)

In [12]:
del no_html_content

In [13]:
content=[s.strip('\n') for s in content]

In [14]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [15]:
unigrams=[]
for s in content:
    unigrams+=tokenizer.tokenize(s)

In [16]:
del content

In [17]:
unigrams=[x for x in map(str.lower,unigrams)]

In [18]:
from collections import Counter
count = Counter(unigrams)
count

Counter({'council': 92126,
         'question': 37196,
         'time': 65400,
         'the': 3382013,
         'next': 20698,
         'item': 7326,
         'is': 741117,
         'b6': 361,
         '0316': 23,
         '2007': 5266,
         'following': 13204,
         'questions': 11416,
         'are': 328584,
         'addressed': 3585,
         'to': 1487107,
         'no': 75037,
         '1': 14846,
         'inadmissible': 284,
         'annex': 1435,
         'ii': 2711,
         'part': 33169,
         'a': 748142,
         'paragraph': 5315,
         '2': 10216,
         'of': 1603555,
         'rules': 23682,
         'procedure': 16108,
         'subject': 18154,
         'prospects': 1854,
         'for': 514893,
         'galileo': 1042,
         'project': 8428,
         'an': 151549,
         'extremely': 11284,
         'important': 66681,
         'european': 264002,
         'union': 117707,
         'as': 315759,
         'it': 417046,
         'result': 17152

In [19]:
 rare= set(key for key, value in count.items() if value <10)

In [20]:
unigrams1=[]
for word in unigrams:
    if word in rare:
        unigrams1.append('*rare*')
    else:
         unigrams1.append(word)

In [22]:
del unigrams

In [23]:
from collections import Counter
count = Counter(unigrams1)

In [24]:
count = {key: value for key, value in count.items() if key!='*rare*'}

In [27]:
from nltk.util import ngrams
bigrams = [ gram for gram in ngrams(unigrams1, 2) ]

In [28]:
rare_bi=[]
for bigram in bigrams:
         if bigram[0]=='*rare*' or bigram[1]=='*rare*':
                       rare_bi.append(bigram)    
    

In [29]:
count1=Counter(bigrams)

In [31]:
for item in rare_bi:
    count1.pop(item, None)

In [33]:
del bigrams

In [34]:
del rare_bi

In [35]:
trigrams = [ gram for gram in ngrams(unigrams1, 3) ]

In [36]:
del unigrams1

In [37]:
rare_tri=[]
for trigram in trigrams:
    if trigram[0]=='*rare*' or trigram[1]=='*rare*' or trigram[2]=='*rare*':
        rare_tri.append(trigram)

In [39]:
count2=Counter(trigrams)

In [40]:
for item in rare_tri:
    count2.pop(item, None)

In [41]:
del trigrams
del rare_tri

In [144]:
#trigram model
def trimodel(tup):
    trikeys,bikeys,unikeys=count2.keys(),count1.keys(),count.keys()
    if tup[0:2] in bikeys:
        if tup in trikeys:
             p=(count2[tup]+1)/(count1[tup[0:2]]+len(count))
        else :
             p=1/(count1[tup[0:2]]+len(count))
    else : 
        if tup[1:] in bikeys:
             p=0.4*count1[tup[1:]]/count[tup[1]]
        else:
            if tup[2] in unikeys:
                 p=0.4*count[tup[2]]/sum(count.values())
            else :
                 p=10**(-15)
    return p     

In [145]:
#bigram model
def bimodel(tup):
    bikeys,unikeys=count1.keys(),count.keys()
    if tup[0] in unikeys:
        if tup in bikeys:
             p=(count1[tup]+1)/(count[tup[0]]+len(count))
        else :
             p=1/(count[tup[0]]+len(count))
    else : 
        if tup[1] in unikeys:
            p=0.4*(count[tup[1]]/sum(count.values()))
        else: 
            p=10**(-15)
    return p    


Αρχη του (iii)

Δημιουργια λίστας με τις προτάσεις των txt files

In [ ]:
content=[]
for file in test_txts:
    fp=open(file, encoding="utf8")
    try:
        for line in fp:
             content.append(line)
    finally:
        fp.close()

Αφαίρεση των htmls

In [46]:
no_html_content=[]
for s in content:
     no_html_content.append(cleanhtml(s))


Aφαίρεση καθε πρόταση η αποτελείται μόνο από '/n' 

In [48]:
content=[]
for s in no_html_content:
    if s!='\n':
        content.append(s)

Aφαίρεση του '/n' από το τέλος της καθε πρότασης

In [49]:
content=[s.strip('\n') for s in content]

In [50]:
del no_html_content

επιλέγουμε τυχαία 100 προτασεις απο το τεστ σετ

In [51]:
random.seed(2)
s=random.sample(range(0,len(content)),100)

In [52]:
list_of_sent=[]
for i in s:
        list_of_sent.append(content[i])

καθε πρόταση την σπάμε σε tokens

In [53]:
list_of_unis=[]
for sent in list_of_sent:
     list_of_unis.append(tokenizer.tokenize(sent))

τα κάνουμε όλα μικρά

In [59]:
for i in range(len(list_of_unis)):
    list_of_unis[i]= [x for x in map(str.lower,list_of_unis[i])]

σε μια λίστα αποθηκευουμε τον αριθμό τον λεξεων της καθε προτασης

In [109]:
list_of_lens=[]
for unis in list_of_unis:
    list_of_lens.append(len(unis))
    

βρισκουμε τα διγράμματα της κάθε πρότασης

In [92]:
list_of_bis=[]
for unis in list_of_unis:
    if len(unis)>1:
        list_of_bis.append([ gram for gram in ngrams(unis, 2)])
    else:
        list_of_bis.append(('*empty*',unis[0]))

βρισκουμε τα τριγράμματα της κάθε πρότασης

In [114]:
list_of_tris=[]
for unis in list_of_unis:
    if len(unis)>2:
         list_of_tris.append([ gram for gram in ngrams(unis, 3)])
    elif len(unis)==2:
        list_of_tris.append(('*empty*',unis[0],unis[1]))
    elif len(unis)==1:
         list_of_tris.append(('*empty*','*empty*',unis[0]))

συνάρτηση η οποία υπολογίζει το αθροισμα των log πιθανοτητων P($w_i$|$w_{i-1}$) παιρνοντας ως input μια προταση σπασμένη σε διγράμματα

In [62]:
import math
def bi_sent_log_prob(bis):
    l_bis=[]
    for bi in bis:
        l_bis.append(math.log(bimodel(bi)))
        return sum(l_bis)

υπολογισμός των πιθανοτητων για τις 100 correct προτάσεις από το τεστ σετ σε διγράμματα

In [162]:
sent_prob=[]
for sent_bis in list_of_bis:
             sent_prob.append(bi_sent_log_prob(sent_bis))
             

μεση πιθανοτητα υπαρξης των 100 correct προτάσεων σε διγράμματα

In [164]:
sum(sent_prob)/len(sent_prob)

-4.749026734654751

συνάρτηση η οποία υπολογίζει το αθροισμα των log πιθανοτητων P($w_i$|$w_{i-1}$,$w_{i-2}$) παιρνοντας ως input μια προταση σπασμένη σε τριγράμματα

In [112]:
def tri_sent_log_prob(tris):
    l_tris=[]
    for tri in tris:
        l_tris.append(math.log(trimodel(tri)))
    return sum(l_tris)

υπολογισμός των πιθανοτητων για τις 100 correct προτάσεις από το τεστ σετ σε τριγράμματα

In [141]:
sent_prob1=[]
for sent_tris in list_of_tris:
    sent_prob1.append(tri_sent_log_prob(sent_tris))


μεση πιθανοτητα υπαρξης των 100 correct προτάσεων σε τριγράμματα

In [142]:
sum(sent_prob1)/len(sent_prob1)

-491.9546435226776

φτιάχνουμε μια λιστα με τα τις distinct words που εμφανίζονται στο train set

In [120]:
list_of_words=[]
for word in count.keys():
    list_of_words.append(word)

συνάρτηση η οποία φτιάχνει μια λιστα απο τυχαιες λέξεις απο μια λιστα λεξεων με συγκεκριμενο αριθμό λέξεων, εστω αυτη η λίστα είναι τα tokens μιας λάθος προτασης 

In [146]:
def create_sent(n_words,l_words):
    s=random.sample(range(0,len(l_words)), n_words)
    l=[]
    for i in s:
        l.append(l_words[i])
    return l    

δημιουργουμε 100 τετοιες λάθος προτάσεις /λιστες tokens

In [147]:
random.seed(2)
l_rand_sent=[]
for i in range(len(list_of_sent)):
    l_rand_sent.append(create_sent(list_of_lens[i],list_of_words))


φτίαχνουμε τα διγράμματα των λάθος προτάσεων

In [148]:
list_of_bis1=[]
for unis in l_rand_sent:
    if len(unis)>1:
        list_of_bis1.append([ gram for gram in ngrams(unis, 2)])
    else:
        list_of_bis.append(('*empty*',unis[0]))

φτίαχνουμε τα τριγράμματα των λάθος προτάσεων

In [151]:
list_of_tris1=[]
for unis in l_rand_sent:
        if len(unis)>2:
            list_of_tris1.append([ gram for gram in ngrams(unis, 3)])
        elif len(unis)==2:
            list_of_tris.append(('*empty*',unis[0],unis[1]))
        elif len(unis)==1:
            list_of_tris.append(('*empty*','*empty*',unis[0]))

υπολογισμός των πιθανοτητων για τις 100 incorrect προτάσεις σε διγράμματα

In [152]:
sent_prob2=[]
for sent_bis in list_of_bis1:
    sent_prob2.append(bi_sent_log_prob(sent_bis))

υπολογισμός των πιθανοτητων για τις 100 incorrect προτάσεις σε τριγράμματα

In [153]:
sent_prob3=[]
for sent_tris in list_of_tris1:
    sent_prob3.append(tri_sent_log_prob(sent_tris))


μεση πιθανοτητα υπαρξης των 100 incorrect προτάσεων σε διγράμματα

In [154]:
sum(sent_prob2)/len(sent_prob2)

-10.290057602647463

μεση πιθανοτητα υπαρξης των 100 incorrect προτάσεων σε τριγράμματα

In [155]:
sum(sent_prob3)/len(sent_prob3)

-964.0050276846106

In [45]:
import numpy as np
np.save('unigrams.npy', count) 

In [46]:
np.save('bigrams.npy', count1) 

In [47]:
np.save('trigrams.npy', count2) 